In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

# root="data"	=> The directory where the dataset will be stored. If "data" doesn’t exist, it will be created.
# train=True =>	This tells PyTorch to load the training set (60,000 images).
# download=True =>	If the dataset is not already present, download it from the internet.
# transform=ToTensor() =>	Apply a transformation that converts each image (PIL format) into a PyTorch tensor and normalizes pixel values to [0.0, 1.0].

100%|██████████| 26.4M/26.4M [00:02<00:00, 10.2MB/s]
100%|██████████| 29.5k/29.5k [00:00<00:00, 170kB/s]
100%|██████████| 4.42M/4.42M [00:01<00:00, 3.21MB/s]
100%|██████████| 5.15k/5.15k [00:00<00:00, 10.7MB/s]


In [3]:
batch_size = 64
# This sets the number of samples per batch.

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)
# This wraps the training_data (which is a FashionMNIST dataset) in a DataLoader.
# What DataLoader does:
  # Batches the data into size batch_size
  # Allows iterating over the dataset using a for loop
  # Optionally shuffles or loads data in parallel (more on this later)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

# output [64, 1, 28, 28]: Each batch contains 64 grayscale images (1 channel, 28x28 pixels) and their 64 labels.

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [4]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")
# Checks if any hardware accelerator (GPU or Apple Silicon, etc.) is available.
# If yes, it uses it (e.g., "cuda" for NVIDIA GPU, "mps" for Apple GPU).
# If not, it defaults to "cpu".

# Define model
class NeuralNetwork(nn.Module):
  # define a class that inherits from nn.Module, which is the base class for all PyTorch models.
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        # nn.Flatten() converts the 2D image (28×28) into a 1D vector (784 values).
        # Shape change: [batch_size, 1, 28, 28] ⟶ [batch_size, 784]
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [5]:
loss_fn = nn.CrossEntropyLoss()
# A loss function measures how far the model’s predictions are from the correct labels (truth). It guides the training process.
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)
# An optimizer updates the model’s weights based on the computed gradients during backpropagation.

In [6]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # This gives you the total number of samples in the dataset (not just in one batch).

    model.train()
    # This sets the model in training mode.
      # It activates dropout, batch norm, etc. if used.
      # Not needed in inference mode (model.eval() is used for testing/validation).

    for batch, (X, y) in enumerate(dataloader):
        # batch: Index of the batch (0, 1, 2, ...)
        # (X, y): A batch of inputs (X) and targets (y) from the dataloader

        # Example shapes:
        #   X: [64, 1, 28, 28]
        #   y: [64]

        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        # Feeds the batch of inputs X through the model.
        # Returns logits (raw output scores) of shape [batch_size, 10].

        loss = loss_fn(pred, y)
        # Compares predictions (pred) with actual labels (y)
        # Returns the average loss for this batch (a single number).

        # Backpropagation
        loss.backward()
        # Calculates the gradient of the loss w.r.t. each model parameter (using the chain rule).
        # These gradients are stored in parameter.grad.

        optimizer.step()
        # Uses the gradients calculated in loss.backward() to update the model's parameters.

        optimizer.zero_grad()
        # Clears the previous gradients, which accumulate by default in PyTorch.
        # Must do this before the next batch.

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")